<a href="https://colab.research.google.com/github/sanjana-joshi14/Ecommerce-Analysis/blob/main/Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
customer_df = pd.read_csv('/content/olist_customers_dataset.csv')
products_df = pd.read_csv('/content/olist_products_dataset.csv')
orders_df = pd.read_csv('/content/olist_orders_dataset.csv')
items_df = pd.read_csv('/content/olist_order_items_dataset.csv')
cat_names_df = pd.read_csv('/content/product_category_name_translation.csv')
payments_df = pd.read_csv('/content/olist_order_payments_dataset.csv')
geos_df = pd.read_csv('/content/olist_geolocation_dataset.csv')
sellers_df = pd.read_csv('/content/olist_sellers_dataset.csv')

In [ ]:
#Merging files to common IDs
full_df = orders_df.merge(payments_df, on='order_id')\
.merge(items_df, on='order_id')\
.merge(customer_df, on='customer_id')\
.merge(products_df, on='product_id')

In [ ]:
# FUnction to summarize columns in the dataset
def df_summary(df):
  summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
  summary = summary.reset_index()
  summary['Column'] = summary['index']
  summary = summary[['Column','dtypes']]
  summary['# unique'] = df.nunique().values
  summary['# duplicates'] = df.duplicated().sum()
  summary['# missing'] = df.isnull().sum().values
  summary['Example'] = df.loc[0].values

  return summary

  df_summary(full_df)

In [ ]:
# Save merged df as CSV
full_df.to_csv('Mergev2.csv', index=False)

In [ ]:
#Importing main DB as MERGED

# Import necessary libraries
from google.cloud import bigquery
from google.oauth2 import service_account # Import for explicit authentication

# Construct a BigQuery client object.
# Explicitly provide credentials using a service account key file
credentials = service_account.Credentials.from_service_account_file(
    '/content/drive/MyDrive/x.json'  # Replace with the actual path
)
client = bigquery.Client(credentials=credentials)


# Define your BigQuery dataset and table.
project_id = "p1-ecomm"
dataset_id = "T1_ECOMM"  # Replace with your dataset ID
table_id = "MERGED"     # Replace with your table ID
table_ref = client.dataset(dataset_id, project=project_id).table(table_id)


# Convert the pandas DataFrame to a BigQuery table.
# Replace 'full_df' with the actual variable name containing your dataframe
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE #replace with WRITE_APPEND if you want to add data
job = client.load_table_from_dataframe(
    full_df, table_ref, job_config=job_config
)
job.result()  # Wait for the job to complete.

print(f"Loaded {full_df.shape[0]} rows into BigQuery table {table_id}")